In [1]:
import pandas as pd

df = pd.read_csv('auto-mpg.csv')

In [2]:
df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight            int64
acceleration    float64
model year        int64
origin            int64
car name         object
dtype: object

In [3]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [4]:
# This will remove duplicate rows from the dataframe
df = df.drop_duplicates()
df.shape

(398, 9)

In [5]:
# This will display the unique values in each column of the dataframe
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Unique values in {column}: {unique_values}, \nCount of unique values in {column}: {len(unique_values)}")

Unique values in mpg: [18.  15.  16.  17.  14.  24.  22.  21.  27.  26.  25.  10.  11.   9.
 28.  19.  12.  13.  23.  30.  31.  35.  20.  29.  32.  33.  17.5 15.5
 14.5 22.5 24.5 18.5 29.5 26.5 16.5 31.5 36.  25.5 33.5 20.5 30.5 21.5
 43.1 36.1 32.8 39.4 19.9 19.4 20.2 19.2 25.1 20.6 20.8 18.6 18.1 17.7
 27.5 27.2 30.9 21.1 23.2 23.8 23.9 20.3 21.6 16.2 19.8 22.3 17.6 18.2
 16.9 31.9 34.1 35.7 27.4 25.4 34.2 34.5 31.8 37.3 28.4 28.8 26.8 41.5
 38.1 32.1 37.2 26.4 24.3 19.1 34.3 29.8 31.3 37.  32.2 46.6 27.9 40.8
 44.3 43.4 36.4 44.6 40.9 33.8 32.7 23.7 23.6 32.4 26.6 25.8 23.5 39.1
 39.  35.1 32.3 37.7 34.7 34.4 29.9 33.7 32.9 31.6 28.1 30.7 24.2 22.4
 34.  38.  44. ], 
Count of unique values in mpg: 129
Unique values in cylinders: [8 4 6 3 5], 
Count of unique values in cylinders: 5
Unique values in displacement: [307.  350.  318.  304.  302.  429.  454.  440.  455.  390.  383.  340.
 400.  113.  198.  199.  200.   97.  110.  107.  104.  121.  360.  140.
  98.  232.  225.  250.  351. 

In [6]:
import numpy as np

# Replace '?' with NaN, convert to float and then replace with median
df['horsepower'].replace('?', np.NaN, inplace=True)
df['horsepower'] = df['horsepower'].astype(float)
median_value = df['horsepower'].median()
df['horsepower'].fillna(median_value, inplace=True)

In [7]:
df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower      float64
weight            int64
acceleration    float64
model year        int64
origin            int64
car name         object
dtype: object

In [8]:
df.drop('car name', axis=1, inplace=True)

In [9]:
# Creating new features
df['displacement_per_cylinder'] = df['displacement'] / df['cylinders']
df['power_to_weight_ratio'] = df['horsepower'] / df['weight']
# Add the new attributes to the dataframe.
df = df[['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'power_to_weight_ratio', 'displacement_per_cylinder']]

In [39]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,power_to_weight_ratio,displacement_per_cylinder
28,9.0,8,304.0,193.0,4732,18.5,70,1,0.040786,38.000
25,10.0,8,360.0,215.0,4615,14.0,70,1,0.046587,45.000
26,10.0,8,307.0,200.0,4376,15.0,70,1,0.045704,38.375
27,11.0,8,318.0,210.0,4382,13.5,70,1,0.047923,39.750
67,11.0,8,429.0,208.0,4633,11.0,72,1,0.044895,53.625


In [10]:
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Unique values in {column}: {unique_values}, \nCount of unique values in {column}: {len(unique_values)}")

Unique values in mpg: [18.  15.  16.  17.  14.  24.  22.  21.  27.  26.  25.  10.  11.   9.
 28.  19.  12.  13.  23.  30.  31.  35.  20.  29.  32.  33.  17.5 15.5
 14.5 22.5 24.5 18.5 29.5 26.5 16.5 31.5 36.  25.5 33.5 20.5 30.5 21.5
 43.1 36.1 32.8 39.4 19.9 19.4 20.2 19.2 25.1 20.6 20.8 18.6 18.1 17.7
 27.5 27.2 30.9 21.1 23.2 23.8 23.9 20.3 21.6 16.2 19.8 22.3 17.6 18.2
 16.9 31.9 34.1 35.7 27.4 25.4 34.2 34.5 31.8 37.3 28.4 28.8 26.8 41.5
 38.1 32.1 37.2 26.4 24.3 19.1 34.3 29.8 31.3 37.  32.2 46.6 27.9 40.8
 44.3 43.4 36.4 44.6 40.9 33.8 32.7 23.7 23.6 32.4 26.6 25.8 23.5 39.1
 39.  35.1 32.3 37.7 34.7 34.4 29.9 33.7 32.9 31.6 28.1 30.7 24.2 22.4
 34.  38.  44. ], 
Count of unique values in mpg: 129
Unique values in cylinders: [8 4 6 3 5], 
Count of unique values in cylinders: 5
Unique values in displacement: [307.  350.  318.  304.  302.  429.  454.  440.  455.  390.  383.  340.
 400.  113.  198.  199.  200.   97.  110.  107.  104.  121.  360.  140.
  98.  232.  225.  250.  351. 

In [11]:
df.sort_values(['mpg', 'cylinders'], inplace=True)
df.head(15)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,power_to_weight_ratio,displacement_per_cylinder
28,9.0,8,304.0,193.0,4732,18.5,70,1,0.040786,38.000
25,10.0,8,360.0,215.0,4615,14.0,70,1,0.046587,45.000
26,10.0,8,307.0,200.0,4376,15.0,70,1,0.045704,38.375
27,11.0,8,318.0,210.0,4382,13.5,70,1,0.047923,39.750
67,11.0,8,429.0,208.0,4633,11.0,72,1,0.044895,53.625
103,11.0,8,400.0,150.0,4997,14.0,73,1,0.030018,50.000
124,11.0,8,350.0,180.0,3664,11.0,73,1,0.049127,43.750
42,12.0,8,383.0,180.0,4955,11.5,71,1,0.036327,47.875
69,12.0,8,350.0,160.0,4456,13.5,72,1,0.035907,43.750
90,12.0,8,429.0,198.0,4952,11.5,73,1,0.039984,53.625


In [40]:
import numpy as np
import pandas as pd

def augment_data(data):
    unique_mpg_values = data['mpg'].unique()
    new_data = []

    for mpg_value in unique_mpg_values:
        instances = data[data['mpg'] == mpg_value]
        instances_sorted = instances.sort_values('cylinders', ascending=True)

        if len(instances_sorted) > 1:
            for i in range(len(instances_sorted) - 1):
                current_instance = instances_sorted.iloc[i]
                next_instance = instances_sorted.iloc[i + 1]

                mean_displacement = np.mean([current_instance['displacement'], next_instance['displacement']])
                mean_horsepower = np.mean([current_instance['horsepower'], next_instance['horsepower']])
                mean_weight = np.int32(np.mean([current_instance['weight'], next_instance['weight']]))
                mean_acceleration = np.mean([current_instance['acceleration'], next_instance['acceleration']])

                # Add noise to the mean values
                mean_displacement += np.random.normal(0, 0.1)
                mean_horsepower += np.random.normal(0, 0.1)
                mean_weight += np.random.normal(0, 10)
                mean_acceleration += np.random.normal(0, 0.1)

                # Round the values to a reasonable precision
                mean_displacement = round(mean_displacement, 2)
                mean_horsepower = round(mean_horsepower, 2)
                mean_weight = int(round(mean_weight))
                mean_acceleration = round(mean_acceleration, 2)

                power_to_weight_ratio = mean_horsepower / mean_weight
                displacement_per_cylinder = mean_displacement / current_instance['cylinders']

                new_instance = {
                    'mpg': current_instance['mpg'],
                    'cylinders': current_instance['cylinders'],
                    'displacement': mean_displacement,
                    'horsepower': mean_horsepower,
                    'weight': mean_weight,
                    'acceleration': mean_acceleration,
                    'model year': current_instance['model year'],
                    'origin': current_instance['origin'],
                    'power_to_weight_ratio': power_to_weight_ratio,
                    'displacement_per_cylinder': displacement_per_cylinder
                }

                new_data.append(new_instance)

    return pd.DataFrame(new_data)

augmented_df = augment_data(df)
print(augmented_df)


      mpg  cylinders  displacement  horsepower  weight  acceleration  \
0    10.0        8.0        333.54      207.60    4497         14.55   
1    11.0        8.0        373.52      209.10    4489         12.05   
2    11.0        8.0        414.61      179.02    4808         12.42   
3    11.0        8.0        375.02      165.04    4319         12.38   
4    12.0        8.0        366.62      169.99    4692         12.44   
..    ...        ...           ...         ...     ...           ...   
264  37.0        4.0        101.87       78.68    2206         17.19   
265  37.0        4.0         88.17       66.29    2001         18.92   
266  38.0        4.0         98.01       65.05    2040         14.88   
267  38.0        4.0         91.01       67.17    1978         15.48   
268  38.0        4.0        176.62       75.86    2504         16.56   

     model year  origin  power_to_weight_ratio  displacement_per_cylinder  
0          70.0     1.0               0.046164             

In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Split the augmented data into features (X_augmented) and target variable (y_augmented)
X_augmented = augmented_df.iloc[ : , 1:]
y_augmented = augmented_df['mpg']

# Split the original data into features (X) and target variable (y)
X = df.iloc[ : , 1:]
y = df['mpg']

# Split the augmented data into training and testing sets
X_train_augmented, X_test_augmented, y_train_augmented, y_test_augmented = train_test_split(
    X_augmented, y_augmented, test_size=0.2, random_state=42
)

# Create and train the linear regression model on augmented data
model = LinearRegression()
model.fit(X_train_augmented, y_train_augmented)

# Test the model on the original data
test_score = model.score(X, y)
print(test_score)


0.839422324683309


In [42]:
from sklearn.preprocessing import StandardScaler

# Combine the augmented and original data
combined_df = pd.concat([augmented_df, df], ignore_index=True)

# Select the columns to be normalized
columns_to_normalize = ['displacement', 'horsepower', 'weight', 'acceleration', 'power_to_weight_ratio', 'displacement_per_cylinder']

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler on the combined data
scaler.fit(combined_df[columns_to_normalize])

# Transform the selected columns
combined_df[columns_to_normalize] = scaler.transform(combined_df[columns_to_normalize])

In [43]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

# Split the combined data into features (X) and target variable (y)
X = combined_df.iloc[:, 1:]
y = combined_df['mpg']

# Split the combined data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Create a dictionary to store the model results
results = {'Model': [], 'Train Accuracy': [], 'Test Accuracy': []}

# Linear Regression
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)
linear_reg_train_acc = linear_reg.score(X_train, y_train)
linear_reg_test_acc = linear_reg.score(X_test, y_test)
results['Model'].append('Linear Regression')
results['Train Accuracy'].append(linear_reg_train_acc)
results['Test Accuracy'].append(linear_reg_test_acc)

# Ridge Regression
ridge_reg = Ridge()
ridge_reg.fit(X_train, y_train)
ridge_reg_train_acc = ridge_reg.score(X_train, y_train)
ridge_reg_test_acc = ridge_reg.score(X_test, y_test)
results['Model'].append('Ridge Regression')
results['Train Accuracy'].append(ridge_reg_train_acc)
results['Test Accuracy'].append(ridge_reg_test_acc)

# Lasso Regression
lasso_reg = Lasso()
lasso_reg.fit(X_train, y_train)
lasso_reg_train_acc = lasso_reg.score(X_train, y_train)
lasso_reg_test_acc = lasso_reg.score(X_test, y_test)
results['Model'].append('Lasso Regression')
results['Train Accuracy'].append(lasso_reg_train_acc)
results['Test Accuracy'].append(lasso_reg_test_acc)

# RandomForestRegressor
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, y_train)
rf_reg_train_acc = rf_reg.score(X_train, y_train)
rf_reg_test_acc = rf_reg.score(X_test, y_test)
results['Model'].append('Random Forest Regressor')
results['Train Accuracy'].append(rf_reg_train_acc)
results['Test Accuracy'].append(rf_reg_test_acc)

# Create a pandas DataFrame from the results dictionary
df_results = pd.DataFrame(results)

# Display the results
print(df_results)

                     Model  Train Accuracy  Test Accuracy
0        Linear Regression        0.869464       0.852618
1         Ridge Regression        0.867914       0.854781
2         Lasso Regression        0.804910       0.811985
3  Random Forest Regressor        0.984789       0.926528


#### Linear Regression:
The Linear Regression model achieved a good fit to the training and test data. It shows a stable performance without significant overfitting.

#### Ridge Regression:
The Ridge Regression model performed similarly to Linear Regression. Ridge regularization helps control overfitting by adding a penalty term.

#### Lasso Regression:
The Lasso Regression model achieved a lower train accuracy and test accuracy. Lasso performs feature selection by driving some coefficients to zero, which may have impacted the model's performance.

#### Random Forest Regressor:
The Random Forest Regressor demonstrated excellent performance, with a higher train accuracy and lower test accuracy . It has the potential to capture complex relationships in the data but may be prone to overfitting.

In [36]:
from sklearn.ensemble import GradientBoostingRegressor

# Random Forest Regressor
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, y_train)
rf_reg_train_acc = rf_reg.score(X_train, y_train)
rf_reg_test_acc = rf_reg.score(X_test, y_test)

# Gradient Boosting Regressor with Random Forest as base estimator
gb_rf_reg = GradientBoostingRegressor(init=rf_reg)
gb_rf_reg.fit(X_train, y_train)
gb_rf_reg_train_acc = gb_rf_reg.score(X_train, y_train)
gb_rf_reg_test_acc = gb_rf_reg.score(X_test, y_test)

# Create a dictionary to store the model results
results = {'Model': ['Random Forest Regressor', 'Gradient Boosting with Random Forest'],
           'Train Accuracy': [rf_reg_train_acc, gb_rf_reg_train_acc],
           'Test Accuracy': [rf_reg_test_acc, gb_rf_reg_test_acc]}

# Create a pandas DataFrame from the results dictionary
df_results = pd.DataFrame(results)

# Display the results
print(df_results)

                                  Model  Train Accuracy  Test Accuracy
0               Random Forest Regressor        0.984459       0.922066
1  Gradient Boosting with Random Forest        0.994235       0.927118


In [44]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

# Define the parameter grid for Gradient Boosting Regressor
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [2, 3, 4],  
    'subsample': [0.7, 0.8, 0.9], 
}

# Define the scoring metric (R-squared)
scorer = make_scorer(lambda y_true, y_pred: np.corrcoef(y_true, y_pred)[0, 1] ** 2)

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    GradientBoostingRegressor(init=rf_reg),
    param_distributions=param_grid,
    n_iter=10,
    scoring=scorer,
    cv=5,
    random_state=42,
)

# Fit the RandomizedSearchCV on the training data
random_search.fit(X_train, y_train)

# Get the best model and its performance
best_model = random_search.best_estimator_
best_model_train_acc = best_model.score(X_train, y_train)
best_model_test_acc = best_model.score(X_test, y_test)

# Print the best hyperparameters and performance
print("Best Hyperparameters:")
print(random_search.best_params_)
print("Train Accuracy:", best_model_train_acc)
print("Test Accuracy:", best_model_test_acc)

Best Hyperparameters:
{'subsample': 0.8, 'n_estimators': 200, 'max_depth': 4, 'learning_rate': 0.01}
Train Accuracy: 0.9909293601236613
Test Accuracy: 0.9286689658918296
